In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install pygaggle before transformers (first create the index using pyserini then install pygaggle)
!pip install pygaggle==0.0.2
!pip install transformers==4.17.0

In [ ]:
import transformers
import pygaggle
import pandas as pd
from tqdm import tqdm
print(transformers.__version__)

4.17.0


In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
reranker =  MonoT5()

#Mono T5 Reranker (1st step)

In [ ]:
#getting the 1500 initial retrievals for each query
relevant_bm25=pd.read_csv("/content/drive/MyDrive/Touche/Final_Relevant_BM25.csv")


In [ ]:
relevant_bm25.head()

In [ ]:
#storing the order of columns
columns_order=relevant_bm25.columns

In [ ]:
#reranking the 1500 documents for each query
new=[]
for title in tqdm(relevant_bm25["title"].unique()):
  passages= relevant_bm25.loc[relevant_bm25["title"]==title]
  texts = [ Text(p[1].content, {'docid': p[1].doc_id}, 0) for p in passages.iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 1500):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [51:01<00:00, 61.22s/it]


In [ ]:
#Creating the reranked file
df=pd.DataFrame(new)
df.head()

,doc_id,Score,title
0,clueweb12-0608wb-66-03868___1,-0.259567,"Which is better, a laptop or a desktop?"
1,clueweb12-0001wb-72-30848___8,-0.938506,"Which is better, a laptop or a desktop?"
2,clueweb12-0006wb-17-30732___8,-6.897573,"Which is better, a laptop or a desktop?"
3,clueweb12-0907wb-60-03588___2,-0.341578,"Which is better, a laptop or a desktop?"
4,clueweb12-1804wb-90-15876___2,-0.318014,"Which is better, a laptop or a desktop?"


In [ ]:
#Making the reranked file in the same format as intial retrieval file
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2.csv")
df=df.join(topics[["Number","Title"]].set_index("Title"), on="title")
df=df.join(relevant_bm25[["doc_id","content"]].drop_duplicates(subset=['doc_id']).set_index("doc_id"), on="doc_id")
df=df.rename(columns={"Number":"title_id","Score":"score"})
df=df[columns_order]
df

,title_id,title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?",clueweb12-0608wb-66-03868___1,-0.259567,desktop laptop tech homeforumscontact hijackt...
1,2,"Which is better, a laptop or a desktop?",clueweb12-0001wb-72-30848___8,-0.938506,desktop replacements usually larger heavier s...
2,2,"Which is better, a laptop or a desktop?",clueweb12-0006wb-17-30732___8,-6.897573,gateway nv affordable offers memory ram bette...
3,2,"Which is better, a laptop or a desktop?",clueweb12-0907wb-60-03588___2,-0.341578,turnaround pricing trends past months whereas...
4,2,"Which is better, a laptop or a desktop?",clueweb12-1804wb-90-15876___2,-0.318014,therefore hard drives laptop usually relative...
...,...,...,...,...,...
74995,100,Should I learn Python or R for data analysis?,clueweb12-0002wb-54-09357___64,-11.245920,younger sediments recorded neogene palynoasse...
74996,100,Should I learn Python or R for data analysis?,clueweb12-0006wb-91-21475___45,-11.102736,wordstat software math stat location communic...
74997,100,Should I learn Python or R for data analysis?,clueweb12-0308wb-70-32537___25,-11.959024,accepted publication october author contribut...
74998,100,Should I learn Python or R for data analysis?,clueweb12-0401wb-46-04650___76,-10.666465,mplus allows analysis cross sectional longitu...


In [ ]:
df.to_csv("/content/drive/MyDrive/Touche/Final_Reranked.csv")

# Mono T5 Reranker- Psuedo Relevance Feedback (2nd step)

In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#getting the 1500 initial retrievals for each new query
relevant_bm25=pd.read_csv("/content/drive/MyDrive/Touche/Final_Relevant_BM25_psuedo_relevance.csv")


In [ ]:
relevant_bm25.head()

,title_id,title,New_title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0907wb-60-03588___2,12.3767,turnaround pricing trends past months whereas...
1,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___1,12.3272,desktop laptop tech homeforumscontact hijackt...
2,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___2,12.1899,laptop computers portable virtual cord image ...
3,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0610wb-81-06832___3,12.0058,may even want contact instructor expectations...
4,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-1904wb-64-15725___1,11.9073,desktop laptop computer donut user login user...


In [ ]:
columns_order=relevant_bm25.columns

In [ ]:
def preprocessing(documents):
  new=[]
  for index, vals in tqdm(documents.iterrows()):
    document = vals['contents']
    # Remove punctuation and numbers
    document = re.sub(r"[^A-Za-z\s]+", " ", document)
    # Remove extra XML query tag
    document = document.replace("query"," ")
    # Remove extra white spaces
    document = re.sub(r"[\s]{2,}", " ", document)
    new.append({'id': vals['id'], 'contents': document})
  new=pd.DataFrame(new)
  return new

In [ ]:
documents= pd.read_json(path_or_buf='/content/drive/MyDrive/Touche/touche-task2-passages-expanded-with-queries.jsonl', lines=True)
documents.head()

,id,contents,chatNoirUrl
0,clueweb12-0000tw-00-14115___1,Do Asian-Americans Face Bias in Admissions at ...,https://chatnoir.eu/cache?uuid=25e04d49-8df7-5...
1,clueweb12-0000tw-00-14115___10,By insisting on upholding affirmative action i...,https://chatnoir.eu/cache?uuid=25e04d49-8df7-5...
2,clueweb12-0000tw-00-14115___11,If you don’t have a rich/famous daddy or mommy...,https://chatnoir.eu/cache?uuid=25e04d49-8df7-5...
3,clueweb12-0000tw-00-14115___12,It’s speculative to have an opinion either way...,https://chatnoir.eu/cache?uuid=25e04d49-8df7-5...
4,clueweb12-0000tw-00-14115___13,Probably 2-3 percent. Such is life. — Kent Cla...,https://chatnoir.eu/cache?uuid=25e04d49-8df7-5...


In [ ]:
preprocessed=preprocessing(documents)
preprocessed

868655it [01:55, 7518.09it/s]


,id,contents
0,clueweb12-0000tw-00-14115___1,Do Asian Americans Face Bias in Admissions at ...
1,clueweb12-0000tw-00-14115___10,By insisting on upholding affirmative action i...
2,clueweb12-0000tw-00-14115___11,If you don t have a rich famous daddy or mommy...
3,clueweb12-0000tw-00-14115___12,It s speculative to have an opinion either way...
4,clueweb12-0000tw-00-14115___13,Probably percent Such is life Kent Clarkfield ...
...,...,...
868650,clueweb12-1914wb-21-07316___5,This form of lighting is natural and is the ty...
868651,clueweb12-1914wb-21-07316___6,This type of lighting is very artificial and n...
868652,clueweb12-1914wb-21-07316___7,For those doing a lot of computer work at nig...
868653,clueweb12-1914wb-21-07316___8,When an incandescent bulb breaks all you have ...


In [ ]:
#reranking the 1500 documents for each new query
new=[]
for title in tqdm(relevant_bm25["New_title"].unique()):
  passages= relevant_bm25.loc[relevant_bm25["New_title"]==title]
  passages=passages.join(preprocessed.set_index("id"), on="doc_id")
  texts = [ Text(p[1].contents, {'docid': p[1].doc_id}, 0) for p in passages.iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 1500):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "New_title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [32:14<00:00, 38.69s/it]


In [ ]:
passages

,title_id,title,New_title,doc_id,score,content,contents
73500,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0000tw-01-02230___1,12.090400,python data analysis library pandas python da...,Python Data Analysis Library pandas Python Dat...
73501,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1408wb-14-08336___4,11.698700,likely efficiently coded python either find i...,They likely won t be efficiently coded in Pyth...
73502,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1509wb-05-28610___12,10.881100,free open source statistics package written s...,R is a free open source statistics package wri...
73503,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0915wb-07-23463___2,10.556800,applicants must exceptional communications sk...,Applicants must have exceptional communication...
73504,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1912wb-61-02443___44,10.487100,fix future details see doctests execute pydoc...,I ll fix this in the future For more details s...
...,...,...,...,...,...,...,...
74995,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1114wb-63-10546___6,5.681596,jonathan murphy aerospace systems design labo...,Jonathan Murphy Aerospace Systems Design Labor...
74996,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0309wb-14-10318___197,5.681100,availability freely available supplementary i...,AVAILABILITY Freely available in Supplementary...
74997,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1606wb-37-10434___100,5.680500,following evaluated user code import testpack...,Then if the following is evaluated in the user...
74998,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1116wb-14-13432___25,5.677200,constructivizing stem robert pondiscio februa...,Constructivizing STEM by Robert Pondiscio Febr...


In [ ]:
#Making the reranked file in the same format as intial retrieval file
df=pd.DataFrame(new)
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2-psuedo_relevance.csv")
df=df.join(topics[["Number","Title","New"]].set_index("New"), on="New_title")
df=df.join(preprocessed.set_index("id"), on="doc_id")
df=df.rename(columns={"Number":"title_id","Score":"score","Title":"title","contents":"content"})
df=df[columns_order]
df

,title_id,title,New_title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0907wb-60-03588___2,-0.014775,There has been a turnaround in pricing trends ...
1,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___1,-0.039840,Desktop or Laptop What the Tech HomeForumsCont...
2,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___2,-0.014093,Laptop computers are portable have the virtual...
3,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0610wb-81-06832___3,-0.063617,You may even want to contact the instructor ab...
4,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-1904wb-64-15725___1,-0.163953,Desktop or laptop computer IT Donut User login...
...,...,...,...,...,...,...
74995,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1114wb-63-10546___6,-10.899497,Jonathan Murphy Aerospace Systems Design Labor...
74996,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0309wb-14-10318___197,-12.700192,AVAILABILITY Freely available in Supplementary...
74997,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1606wb-37-10434___100,-13.007623,Then if the following is evaluated in the user...
74998,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-1116wb-14-13432___25,-12.579445,Constructivizing STEM by Robert Pondiscio Febr...


In [ ]:
df.to_csv("/content/drive/MyDrive/Touche/Final_Reranked_psuedo_relevance.csv",index=False)